In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=bce8a87485d3e621728bbada9d967350a8262d042044f8797ee0fa907d6af3e7
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/MyDrive/Projects/Plant_box/work_directory

/content/drive/MyDrive/Projects/Plant_box/work_directory


In [10]:
ls

 areal_summ.xlsx
 clean_data2_sumA.parquet.gzip
 clean_data.parquet
 merged_df.csv
 merged_df.parquet
 merged_df.zip
 plants.txt
 region_herb.csv
 regions.txt
 Атлас_2021.txt
'Дикорастущие лекарственные растения СССР.txt'
 Лекарственные_и_эфирномасличные_растения_Маланкина_Е_,_Цицилин_А.txt
 Пайплайн.docx


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_regions(url):
  # URL страницы для парсинга
  # Отправка GET-запроса и получение содержимого страницы
  response = requests.get(url)
  content = response.content

  # Создание объекта BeautifulSoup для парсинга страницы
  soup = BeautifulSoup(content, 'html.parser')

  # Нахождение всех элементов с тегом "row-lined row-state-normal"
  elements = soup.find_all(class_='row-lined row-state-normal')

  region_dict = {}
  # Перебор найденных элементов
  for element in elements:
      # Нахождение тегов "a" внутри текущего элемента
      a_tag = element.find('a')
      # for td in tds:
      #   print(td.get_text())
      # text = element.get_text()
      # print(text)
      region_dict[a_tag.get_text()] =  'https://www.plantarium.ru/' + a_tag['href']

  return region_dict

def get_list_of_herbs(url):
  # Отправляем GET-запрос и получаем HTML-страницу
  response = requests.get(url)
  html = response.text

  # Создаем объект BeautifulSoup для парсинга HTML
  soup = BeautifulSoup(html, "html.parser")

  # Ищем элементы с тегом "row-state-normal" и "row-lined"
  elements = soup.find_all(class_="row-state-normal row-lined")

  # Перебираем найденные элементы и выводим текст
  list_of_herbs = []
  for element in elements:
      tds = element.find_all('td')
      # print(tds[3].get_text().split(','))
      for i in tds[3].get_text().split(','):
        # print(i.strip())
        list_of_herbs.append(i.strip().lower())
      # for td in tds:
      #   print(td.get_text())
      # text = element.get_text()
      # print(text)
  return list_of_herbs



In [ ]:
#загружаем в словарь списки растений по регионам и записываем в area_herb: ключ наименование округа, значение - список растений
area_herb = {}
region_dict = get_regions('https://www.plantarium.ru/page/redbooks.html')
#список регионов из справочника. обратить вниамние, что наименования официальные могут не совпадать с наименованиями в списке красной книги
my_file = open("regions.txt", "r", encoding = 'utf8')
regions_list = my_file.readlines()
regions_list = [word.strip() for word in regions_list]
my_file.close()
for region in get_regions('https://www.plantarium.ru/page/redbooks.html'):
  if region in regions_list:
    print(region)
    area_herb[region] = get_list_of_herbs(region_dict[region])
  else:
    print(region + " [отсутствует в списке]")


Республика Адыгея
Азербайджанская Республика [отсутствует в списке]
Республика Алтай
Алтайский край
Амурская область
Республика Армения [отсутствует в списке]
Архангельская область
Астраханская область
Балтийский регион [отсутствует в списке]
Республика Башкортостан
Республика Беларусь [отсутствует в списке]
Белгородская область
Республика Болгария [отсутствует в списке]
Брянская область
Республика Бурятия
Бурятская АССР [отсутствует в списке]
Владимирская область
Волгоградская область
Вологодская область
Воронежская область
Восточная Фенноскандия [отсутствует в списке]
Грузинская ССР [отсутствует в списке]
Республика Дагестан
Донецкая Народная Республика [отсутствует в списке]
Еврейская автономная область
Забайкальский край
Ивановская область
Республика Ингушетия
Иркутская область
Кабардино-Балкарская Республика
Казахская ССР [отсутствует в списке]
Республика Казахстан [отсутствует в списке]
Калининградская область
Республика Калмыкия
Калужская область
Камчатский край
Карачаево-Черкес

In [ ]:
# #Собираем по регионам все растения и сохраняем в файл
# for ind, reg in enumerate(get_regions('https://www.plantarium.ru/page/redbooks.html')):
#   print(reg, region_dict[reg], ind)

#   with open(f"regions/{reg}.txt", 'w', encoding='utf-8' ) as f:
#     lines = get_list_of_herbs(region_dict[reg])
#     for line in lines:
#       f.write(f"{line}\n")

In [ ]:
#Загружаем список растений для сравнения
file = open("plants.txt", "r", encoding = 'utf8')
my_herbs_list = file.readlines()
my_herbs_list = [word.strip().lower() for word in my_herbs_list]
file.close()
# Проверяем наличие растений в списке каждого региона
print(my_herbs_list)

['аденостилес ромболистный', 'аир обыкновенный', 'алоэ древовидное', 'алтей лекарственный', 'амми большая', 'амми зубная', 'анис обыкновенный', 'аралия высокая', 'арника горная', 'арника шамиссо', 'арония черноплодная', 'астрагал шерстистоцветковый', 'багульник болотный', 'бадан толстолистный', 'баранец обыкновенный', 'барбарис обыкновенный', 'барвинок малый', 'бархат амурский', 'бедренец камнеломка', 'безвременник великолепный', 'белена черная', 'белокопытник гибридный', 'берёза повислая', 'бессмертник песчаный', 'борец северный', 'боярышник кроваво-красный', 'бриофиллум перистый', 'брусника обыкновенная', 'бузина черная', 'валериана лекарственная', 'василек благословенный', 'василек посевной', 'вахта трехлистная', 'вербена лекарственная', 'вздутоплодник сибирский', 'виноград культурный', 'водосбор обыкновенный', 'володушка золотистая', 'гамамелис виргинский', 'гармала обыкновенная', 'горец змеиный', 'горец перечный', 'горец почечуйный', 'горец птичий', 'горечавка жёлтая', 'горицвет в

In [ ]:
import csv

with open("region_herb.csv", mode="w", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter = "\t")
    file_writer.writerow(["Растение", "Регион"])
    #сравниваем список растений из файла со списком в словаре
    for herb in my_herbs_list:
        for reg in area_herb:
          if herb in area_herb[reg]:
            print(herb + "; " + reg )
            file_writer.writerow([herb + ';', reg])


аир обыкновенный; Республика Марий Эл
аир обыкновенный; Республика Тыва
аир обыкновенный; Тюменская область
алтей лекарственный; Республика Башкортостан
алтей лекарственный; Брянская область
алтей лекарственный; Республика Калмыкия
алтей лекарственный; Кемеровская область
алтей лекарственный; Курганская область
алтей лекарственный; Курская область
алтей лекарственный; Липецкая область
алтей лекарственный; Омская область
алтей лекарственный; Пензенская область
алтей лекарственный; Тамбовская область
алтей лекарственный; Республика Татарстан
алтей лекарственный; Удмуртская республика
алтей лекарственный; Ульяновская область
алтей лекарственный; Чувашская республика
аралия высокая; Сахалинская область
астрагал шерстистоцветковый; Белгородская область
астрагал шерстистоцветковый; Волгоградская область
астрагал шерстистоцветковый; Воронежская область
астрагал шерстистоцветковый; Республика Калмыкия
астрагал шерстистоцветковый; Курская область
астрагал шерстистоцветковый; Липецкая область
ас